In [0]:
data=['ama','bnb']

In [0]:
resturant_json_data=spark.read.format('json')\
          .option('inferschema','True')\
          .option('multiline','True')\
          .load('/FileStore/tables/resturant_json_data.json')
          
          

In [0]:
resturant_json_data.show()

+----+-------+--------------------+-------------+-------------+-------------+------+
|code|message|         restaurants|results_found|results_shown|results_start|status|
+----+-------+--------------------+-------------+-------------+-------------+------+
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17066603}, b9...|         6835|           20|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17093124}, b9...|         8680|           20|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17580142}, b9...|          943|           20|            1|  null|
|null|   null|                  []|            0|            0|  

In [0]:
resturant_json_data.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- restaurants: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restaurant: struct (nullable = true)
 |    |    |    |-- R: struct (nullable = true)
 |    |    |    |    |-- res_id: long (nullable = true)
 |    |    |    |-- apikey: string (nullable = true)
 |    |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |    |-- cuisines: string (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- deeplink: string (nullable = true)
 |    |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- events_url: string (nullable = true)
 |    |    |    |-- featured_image: string (nullable = true)
 |    |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |    |-- has_table_booking: long (nullable = true)
 |    |    |    |-- i

In [0]:
resturant_json_data.select('restaurants')

Out[16]: DataFrame[restaurants: array<struct<restaurant:struct<R:struct<res_id:bigint>,apikey:string,average_cost_for_two:bigint,cuisines:string,currency:string,deeplink:string,establishment_types:array<string>,events_url:string,featured_image:string,has_online_delivery:bigint,has_table_booking:bigint,id:string,is_delivering_now:bigint,location:struct<address:string,city:string,city_id:bigint,country_id:bigint,latitude:string,locality:string,locality_verbose:string,longitude:string,zipcode:string>,menu_url:string,name:string,offers:array<string>,photos_url:string,price_range:bigint,switch_to_order_menu:bigint,thumb:string,url:string,user_rating:struct<aggregate_rating:string,rating_color:string,rating_text:string,votes:string>>>>]

In [0]:
resturant_json_data.select('*',explode_outer('restaurants').alias('new_restaurant')).drop('restaurants').select('*','new_restaurant.restaurant.R.res_id').select('*',explode_outer('new_restaurant.restaurant.establishment_types').alias('new_establishment_types')).select('*',col('new_restaurant.restaurant.name').alias('new_name')).drop('new_restaurant').show()

+----+-------+-------------+-------------+-------------+------+--------+-----------------------+--------------------+
|code|message|results_found|results_shown|results_start|status|  res_id|new_establishment_types|            new_name|
+----+-------+-------------+-------------+-------------+------+--------+-----------------------+--------------------+
|null|   null|            0|            0|            1|  null|    null|                   null|                null|
|null|   null|         6835|           20|            1|  null|17066603|                   null|            The Coop|
|null|   null|         6835|           20|            1|  null|17059541|                   null|Maggiano's Little...|
|null|   null|         6835|           20|            1|  null|17064405|                   null|Tako Cheena by Po...|
|null|   null|         6835|           20|            1|  null|17057797|                   null|Bosphorous Turkis...|
|null|   null|         6835|           20|            1|

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:

customer_dim_data = [

(1,'manish','arwal','india','N','2022-09-15','2022-09-25'),
(2,'vikash','patna','india','Y','2023-08-12',None),
(3,'nikita','delhi','india','Y','2023-09-10',None),
(4,'rakesh','jaipur','india','Y','2023-06-10',None),
(5,'ayush','NY','USA','Y','2023-06-10',None),
(1,'manish','gurgaon','india','Y','2022-09-25',None),
]

customer_schema= ['id','name','city','country','active','effective_start_date','effective_end_date']

customer_dim_df = spark.createDataFrame(data= customer_dim_data,schema=customer_schema)

sales_data = [

(1,1,'manish','2023-01-16','gurgaon','india',380),
(77,1,'manish','2023-03-11','bangalore','india',300),
(12,3,'nikita','2023-09-20','delhi','india',127),
(54,4,'rakesh','2023-08-10','jaipur','india',321),
(65,5,'ayush','2023-09-07','mosco','russia',765),
(89,6,'rajat','2023-08-10','jaipur','india',321)
]

sales_schema = ['sales_id', 'customer_id','customer_name', 'sales_date', 'food_delivery_address','food_delivery_country', 'food_cost']

sales_df = spark.createDataFrame(data=sales_data,schema=sales_schema)

In [0]:
customer_dim_df.show()

+---+------+-------+-------+------+--------------------+------------------+
| id|  name|   city|country|active|effective_start_date|effective_end_date|
+---+------+-------+-------+------+--------------------+------------------+
|  1|manish|  arwal|  india|     N|          2022-09-15|        2022-09-25|
|  2|vikash|  patna|  india|     Y|          2023-08-12|              null|
|  3|nikita|  delhi|  india|     Y|          2023-09-10|              null|
|  4|rakesh| jaipur|  india|     Y|          2023-06-10|              null|
|  5| ayush|     NY|    USA|     Y|          2023-06-10|              null|
|  1|manish|gurgaon|  india|     Y|          2022-09-25|              null|
+---+------+-------+-------+------+--------------------+------------------+



In [0]:
sales_df.show()

+--------+-----------+-------------+----------+---------------------+---------------------+---------+
|sales_id|customer_id|customer_name|sales_date|food_delivery_address|food_delivery_country|food_cost|
+--------+-----------+-------------+----------+---------------------+---------------------+---------+
|       1|          1|       manish|2023-01-16|              gurgaon|                india|      380|
|      77|          1|       manish|2023-03-11|            bangalore|                india|      300|
|      12|          3|       nikita|2023-09-20|                delhi|                india|      127|
|      54|          4|       rakesh|2023-08-10|               jaipur|                india|      321|
|      65|          5|        ayush|2023-09-07|                mosco|               russia|      765|
|      89|          6|        rajat|2023-08-10|               jaipur|                india|      321|
+--------+-----------+-------------+----------+---------------------+-------------